# **Pip Install**

In [1]:
!pip install scikit-learn pandas numpy torch transformers pillow joblib emoji regex diffusers["torch"]

# **Google Drive Initialize**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# **Import Library**

In [3]:
import json
import pandas as pd
import numpy as np
import tqdm
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, f1_score, precision_score, recall_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.model_selection import train_test_split
import torch
import math
import torch.nn as nn
from torch.utils.data import DataLoader, WeightedRandomSampler
from transformers import CLIPProcessor, CLIPModel, AutoTokenizer, AutoModel
import torchvision.transforms as torch_transforms
import torch.nn.functional as F
from torchvision.transforms.functional import InterpolationMode
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torch.serialization as torch_serialization
from PIL import Image
import os
import joblib
import string
import re
import random
from emoji import is_emoji
from collections import Counter
from diffusers import AutoencoderKL, UNet2DConditionModel, DDPMScheduler, StableDiffusionPipeline, EulerDiscreteScheduler

# **Result Ouput Directory**

In [4]:
output_dir = '/content/drive/MyDrive/Prop2Hate-Meme/Result/Elunet/WithOut_Diff'  # Directory to save metrics and predictions
os.makedirs(output_dir, exist_ok=True)

# **Text Preprecessing**

In [5]:
# 1. Load the Datasets
def load_jsonl(file_path):
    data = []
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    return pd.DataFrame(data)

In [6]:
train_df = load_jsonl('/content/drive/MyDrive/Prop2Hate-Meme/arabic_hateful_meme_train.jsonl')
dev_df = load_jsonl('/content/drive/MyDrive/Prop2Hate-Meme/arabic_hateful_meme_dev.jsonl')
test_df = load_jsonl('/content/drive/MyDrive/Prop2Hate-Meme/arabic_hateful_meme_test.jsonl')

In [7]:
# Add 'data/' prefix to full ID path
test_df['id_normalized'] = test_df['id'].apply(lambda x: f"data/{x}")

In [8]:
test_df

,id,text,img_path,hate_label,id_normalized
0,data/arabic_memes_fb_insta_pinterest/Pinterest...,لما اصحي الصبح مليش نفس افطر .\nجايلكوا يشويه ...,./data/arabic_memes_fb_insta_pinterest/Pintere...,0,data/data/arabic_memes_fb_insta_pinterest/Pint...
1,data/arabic_memes_fb_insta_pinterest/Instagram...,اكثد مخلوقين كسلآ فر العالم ينظران الى بعضهما\...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,data/data/arabic_memes_fb_insta_pinterest/Inst...
2,data/arabic_memes_fb_insta_pinterest/Instagram...,شايف خلق جديده في المسجد 7٥ SARCAS SOGB\nواضح ...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,data/data/arabic_memes_fb_insta_pinterest/Inst...
3,data/arabic_memes_fb_insta_pinterest/Instagram...,امي لمت كل الهدوم اللي في البيت عشان تغسلها وم...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,data/data/arabic_memes_fb_insta_pinterest/Inst...
4,data/arabic_memes_fb_insta_pinterest/Instagram...,Hossafassaم\nانا مهربشي 0\nالف جثبج\nكم كومنت ...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,data/data/arabic_memes_fb_insta_pinterest/Inst...
...,...,...,...,...,...
601,data/arabic_memes_fb_insta_pinterest/Instagram...,- يابنتي اللي في سنك فاتحين بيوت وعندهم عيال !...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,data/data/arabic_memes_fb_insta_pinterest/Inst...
602,data/arabic_memes_fb_insta_pinterest/Pinterest...,صور فتيات الليل فى كندا سنة 1940\n605\n3٤\n05م...,./data/arabic_memes_fb_insta_pinterest/Pintere...,1,data/data/arabic_memes_fb_insta_pinterest/Pint...
603,data/arabic_memes_fb_insta_pinterest/Instagram...,-أنتي بتروحي تعملي إيه ف الكليه \nأنا :,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,data/data/arabic_memes_fb_insta_pinterest/Inst...
604,data/arabic_memes_fb_insta_pinterest/Pinterest...,الدكتور قاله ابعد عن الشيشة,./data/arabic_memes_fb_insta_pinterest/Pintere...,0,data/data/arabic_memes_fb_insta_pinterest/Pint...


In [9]:
# prompt: count number of data in train_df

print(f'Train dataset size: {len(train_df)}')
print(f'Test dataset size: {len(test_df)}')
print(f'Dev dataset size: {len(dev_df)}')

Train dataset size: 2143
Test dataset size: 606
Dev dataset size: 312


In [10]:
test_df.head(10)

,id,text,img_path,hate_label,id_normalized
0,data/arabic_memes_fb_insta_pinterest/Pinterest...,لما اصحي الصبح مليش نفس افطر .\nجايلكوا يشويه ...,./data/arabic_memes_fb_insta_pinterest/Pintere...,0,data/data/arabic_memes_fb_insta_pinterest/Pint...
1,data/arabic_memes_fb_insta_pinterest/Instagram...,اكثد مخلوقين كسلآ فر العالم ينظران الى بعضهما\...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,data/data/arabic_memes_fb_insta_pinterest/Inst...
2,data/arabic_memes_fb_insta_pinterest/Instagram...,شايف خلق جديده في المسجد 7٥ SARCAS SOGB\nواضح ...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,data/data/arabic_memes_fb_insta_pinterest/Inst...
3,data/arabic_memes_fb_insta_pinterest/Instagram...,امي لمت كل الهدوم اللي في البيت عشان تغسلها وم...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,data/data/arabic_memes_fb_insta_pinterest/Inst...
4,data/arabic_memes_fb_insta_pinterest/Instagram...,Hossafassaم\nانا مهربشي 0\nالف جثبج\nكم كومنت ...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,data/data/arabic_memes_fb_insta_pinterest/Inst...
5,data/arabic_memes_fb_insta_pinterest/Pinterest...,وانا في السجن بعد ما قتلت وزير التعليم,./data/arabic_memes_fb_insta_pinterest/Pintere...,1,data/data/arabic_memes_fb_insta_pinterest/Pint...
6,data/arabic_memes_fb_insta_pinterest/Instagram...,*لما يكلمك يصالحك بعد ما بدأتي تكراشي ع حد احل...,./data/arabic_memes_fb_insta_pinterest/Instagr...,1,data/data/arabic_memes_fb_insta_pinterest/Inst...
7,data/arabic_memes_fb_insta_pinterest/Facebook/...,عندما يصرح أحد المعاقين ذهنيا أن العلمانية أفض...,./data/arabic_memes_fb_insta_pinterest/Faceboo...,1,data/data/arabic_memes_fb_insta_pinterest/Face...
8,data/arabic_memes_fb_insta_pinterest/Instagram...,«غمض عينيه من اللي بيحصله,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,data/data/arabic_memes_fb_insta_pinterest/Inst...
9,data/arabic_memes_fb_insta_pinterest/Instagram...,_الصحاب لما يجو يتصورو بيتصورو صور حلو ٥ *انا ...,./data/arabic_memes_fb_insta_pinterest/Instagr...,0,data/data/arabic_memes_fb_insta_pinterest/Inst...


In [11]:
# 3. Function to Count and Identify Special Characters
def count_special_chars(text_series, dataset_name, stage):
    # Define special characters: punctuation, emojis, and non-alphanumeric (excluding Arabic)
    arabic_chars = r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF]'  # Arabic Unicode range
    special_chars = []
    for text in text_series:
        for char in text:
            # Check if char is not alphanumeric, not Arabic, and either punctuation or emoji
            if (not char.isalnum() and not re.match(arabic_chars, char) and
                (char in string.punctuation or is_emoji(char))):
                special_chars.append(char)

    # Count occurrences and unique characters
    char_counts = Counter(special_chars)
    total_count = sum(char_counts.values())
    unique_chars = sorted(char_counts.keys())

    # Save results to file
    with open(os.path.join(output_dir, f'{dataset_name}_special_chars_{stage}.txt'), 'w', encoding='utf-8') as f:
        f.write(f"{dataset_name} Special Characters ({stage} cleaning):\n")
        f.write(f"Total Special Characters: {total_count}\n")
        f.write(f"Unique Special Characters: {unique_chars}\n")
        f.write("\nCharacter Counts:\n")
        for char, count in char_counts.items():
            f.write(f"'{char}': {count}\n")

    print(f"\n{dataset_name} ({stage} cleaning):")
    print(f"Total Special Characters: {total_count}")
    print(f"Unique Special Characters: {unique_chars}")

    return total_count, unique_chars, char_counts

In [12]:
# 2. Text Cleaning Function
def preprocess_text(text):
    # Define punctuation to remove (excluding emoji-related characters)
    punctuation = string.punctuation.replace(':', '')  # Preserve colons for emoji handling in CLIP
    # Create a regex pattern to match punctuation, excluding emojis
    pattern = r'[{}]'.format(re.escape(punctuation))
    # Remove punctuation while preserving Arabic text and emojis
    cleaned_text = re.sub(pattern, '', text)
    # Remove excessive whitespace
    cleaned_text = ' '.join(cleaned_text.strip().split())
    # Verify emojis are preserved
    if any(is_emoji(c) for c in cleaned_text):
        print(f"Emojis preserved in: {cleaned_text}")
    return cleaned_text

# Analyze Special Characters Before Cleaning
print("Analyzing special characters before cleaning...")
count_special_chars(train_df['text'], 'train', 'before')
count_special_chars(dev_df['text'], 'dev', 'before')
count_special_chars(test_df['text'], 'test', 'before')

# Apply Text Cleaning to All Datasets
train_df['processed_text'] = train_df['text'].apply(preprocess_text)
dev_df['processed_text'] = dev_df['text'].apply(preprocess_text)
test_df['processed_text'] = test_df['text'].apply(preprocess_text)

# Analyze Special Characters After Cleaning
print("\nAnalyzing special characters after cleaning...")
count_special_chars(train_df['processed_text'], 'train', 'after')
count_special_chars(dev_df['processed_text'], 'dev', 'after')
count_special_chars(test_df['processed_text'], 'test', 'after')

Analyzing special characters before cleaning...

train (before cleaning):
Total Special Characters: 3012
Unique Special Characters: ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '✋', '💔', '🔞', '😂', '😅', '😪', '😳', '🙄']

dev (before cleaning):
Total Special Characters: 343
Unique Special Characters: ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '=', '>', '?', '@', ']', '_', '~', '🤯']

test (before cleaning):
Total Special Characters: 832
Unique Special Characters: ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '?', '@', '[', ']', '^', '_', '`', '|', '}', '🌈', '🏳']
Emojis preserved in: زوجة ماكرون تصرح أن الحجاب يرعب ويخيف الأطفال😅😂😂
Emojis preserved in: انا حاليًا بعيش فتره اسمها انا فيا اللي مكفيني وبتعامل مع الناس والعلاقات بمبدأ كونوا زي ما تكو نوا بس المهم بعيد عني✋️
Emojis preserve

(123, [':', '🌈', '🏳'], Counter({':': 121, '🏳': 1, '🌈': 1}))

In [13]:
print(test_df['processed_text'].to_list())

['لما اصحي الصبح مليش نفس افطر جايلكوا يشويه رفيعين', 'اكثد مخلوقين كسلآ فر العالم ينظران الى بعضهما Mactتyun 8', 'شايف خلق جديده في المسجد 7٥ SARCAS SOGB واضح ان الامتحانات جايه شديحة', 'امي لمت كل الهدوم اللي في البيت عشان تغسلها ومفاضلش غيرالهدوم اللي انا لابسها عشان تحقق التارجيت بتاع الغساله اقلع', 'Hossafassaم انا مهربشي 0 الف جثبج كم كومنت وناخدها ببلاش ؟', 'وانا في السجن بعد ما قتلت وزير التعليم', 'لما يكلمك يصالحك بعد ما بدأتي تكراشي ع حد احلى ده انت غتت', 'عندما يصرح أحد المعاقين ذهنيا أن العلمانية أفضل حل لمواجهة الإرهاب والتطرف وأنها جائت لمحاربة التعصب الديني والصراع بين الأديان جرائم أتاتورك', '«غمض عينيه من اللي بيحصله', 'الصحاب لما يجو يتصورو بيتصورو صور حلو ٥ انا وصحابى لما نيج نتصور', 'لما تسرح بخيالك وانت بتآكل', 'بالله كيف عرفت اني طالب مصري في كليه هندسة', 'الناس الطبيعيه وهي مسافره انا', 'منهم لله الفراعنة فضلوا يرموا الحلوين فى النيل وسابولنا دول', 'بسجل فويس ممكن محد يتكلم ؟ أهلي :', 'الكمامه ال هتحمينا م الفيروس او هتجيبه مش متاكد', 'لا مؤاخذة فى السؤال لامؤااا

In [14]:
# Filter the DataFrame to count rows where 'hate_label' is 1
hate_count = train_df[train_df['hate_label'] == 0].shape[0]
print(f"Number of rows with hate_label == 1: {hate_count}")

Number of rows with hate_label == 1: 1930


# ***SVM***

In [ ]:
# 2. Initialize CLIP model and processor
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
clip_processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
# 4. Extract CLIP Features with Logging
def get_clip_features(text, img_path, meme_id):
  base_dir = '/content/drive/MyDrive/Prop2Hate-Meme'  # Update with actual path
  img_path = os.path.join(base_dir, img_path.lstrip('./'))
  print(f"Processing ID: {meme_id}, Text: {text}")
  try:
      img = Image.open(img_path).convert('RGB')
      inputs = clip_processor(text=[text], images=[img], return_tensors="pt", padding=True, truncation=True).to(device)
      with torch.no_grad():
          outputs = clip_model(**inputs)
      text_embedding = outputs.text_embeds.cpu().numpy()
      image_embedding = outputs.image_embeds.cpu().numpy()
      return np.concatenate([text_embedding, image_embedding], axis=1)[0]
  except Exception as e:
      print(f"Error processing ID: {meme_id}, Image: {img_path}: {e}")
      with open('/content/drive/MyDrive/Prop2Hate-Meme/error_log.txt', 'a', encoding='utf-8') as f:
          f.write(f"Error processing ID: {meme_id}, Image: {img_path}: {e}\n")
      return np.zeros(1024)

In [ ]:
# Extract features for all datasets
print("Extracting features for training set...")
train_features = np.array([get_clip_features(text, img_path, meme_id)
                          for text, img_path, meme_id in zip(train_df['processed_text'], train_df['img_path'], train_df['id'])])
print("\nExtracting features for dev set...")
dev_features = np.array([get_clip_features(text, img_path, meme_id)
                         for text, img_path, meme_id in zip(dev_df['processed_text'], dev_df['img_path'], dev_df['id'])])
print("\nExtracting features for test set...")
test_features = np.array([get_clip_features(text, img_path, meme_id)
                          for text, img_path, meme_id in zip(test_df['processed_text'], test_df['img_path'], test_df['id'])])

In [ ]:
# 5. Labels
train_labels = train_df['hate_label'].values
dev_labels = dev_df['hate_label'].values
test_labels = test_df['hate_label'].values

In [ ]:
# 6. Train SVM Classifier
svm = SVC()
svm.fit(train_features, train_labels)

In [ ]:
# 7. Evaluate Metrics Function
def evaluate_metrics(true_labels, pred_labels, dataset_name):
    micro_f1 = f1_score(true_labels, pred_labels, average='micro')
    precision = precision_score(true_labels, pred_labels, average='micro')
    recall = recall_score(true_labels, pred_labels, average='micro')
    accuracy = accuracy_score(true_labels, pred_labels)

    print(f"\n{dataset_name} Set Evaluation:")
    print(f"Micro-F1 Score (Official Metric): {micro_f1:.4f}")
    print(f"Micro Precision: {precision:.4f}")
    print(f"Micro Recall: {recall:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"{dataset_name} Set Classification Report:\n", classification_report(true_labels, pred_labels))

    # Save metrics to file in output directory
    with open(os.path.join(output_dir, f'{dataset_name}_metrics.txt'), 'w', encoding='utf-8') as f:
        f.write(f"{dataset_name} Set Evaluation:\n")
        f.write(f"Micro-F1 Score (Official Metric): {micro_f1:.4f}\n")
        f.write(f"Micro Precision: {precision:.4f}\n")
        f.write(f"Micro Recall: {recall:.4f}\n")
        f.write(f"Accuracy: {accuracy:.4f}\n")
        f.write(f"\nClassification Report:\n{classification_report(true_labels, pred_labels)}")

    return micro_f1, precision, recall, accuracy

In [ ]:
# 8. Evaluate on Dev and Test Sets
dev_pred = svm.predict(dev_features)
evaluate_metrics(dev_labels, dev_pred, "Dev")

test_pred = svm.predict(test_features)
evaluate_metrics(test_labels, test_pred, "Test")

# **ELUnet**

In [15]:
# make run output folder in google drive
LOG_DIR = "/content/drive/MyDrive/Prop2Hate-Meme/Result/Elunet"

name = "hateful_meme_drive_50_500_samples_bicubic_l1_256_task3"
run_folder = os.path.join(LOG_DIR, "500_hateful_meme", name)

os.makedirs(run_folder, exist_ok=True)

print(f'Run folder: {run_folder}')

Run folder: /content/drive/MyDrive/Prop2Hate-Meme/Result/Elunet/500_hateful_meme/hateful_meme_drive_50_500_samples_bicubic_l1_256_task3


# **ELUnet Model Initialize**

In [16]:
class DoubleConv(nn.Module):
    """ [(Conv2d) => (BN) => (ReLu)] * 2 """

    def __init__(self,in_channels,out_channels) -> None:
        super().__init__()
        self.double_conv = nn.Sequential(
                nn.Conv2d(in_channels,out_channels,3,padding="same",stride=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(),
                nn.Conv2d(out_channels,out_channels,3,padding="same",stride=1),
                nn.BatchNorm2d(out_channels),
                nn.ReLU(),
            )
    def forward(self,x):
        return self.double_conv(x)

class DownSample(nn.Module):

    """ MaxPool => DoubleConv """
    def __init__(self,in_channels,out_channels) -> None:
        super().__init__()
        self.down_sample = nn.Sequential(
            nn.MaxPool2d(2),
            DoubleConv(in_channels,out_channels)
        )

    def forward(self, x):
        x  = self.down_sample(x)
        return x

class UpSample(nn.Module):
    def __init__(self,in_channels,out_channels,c:int) -> None:
        """ UpSample input tensor by a factor of `c`
                - the value of base 2 log c defines the number of upsample
                layers that will be applied
        """
        super().__init__()
        n = 0 if c == 0 else int(math.log(c,2))

        self.upsample = nn.ModuleList(
            [nn.ConvTranspose2d(in_channels,in_channels,2,2) for i in range(n)]
        )
        self.conv_3 = nn.Conv2d(in_channels,out_channels,3,padding="same",stride=1)

    def forward(self, x):
        for layer in self.upsample:
            x = layer(x)
        return self.conv_3(x)

class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size=1)
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.sigmoid = nn.Sigmoid()
    def forward(self, x):
        x = self.conv(x)
        x = self.pool(x)
        x = x.squeeze(-1).squeeze(-1)
        x = self.sigmoid(x)
        return x

In [17]:
def cross_attention(Q, K, V):
    # Compute the dot products between Q and K, then scale
    d_k = Q.size(-1)
    scores = torch.matmul(Q, K.transpose(-2, -1)) / torch.sqrt(torch.tensor(d_k, dtype=torch.float32))

    # Softmax to normalize scores and get attention weights
    attention_weights = F.softmax(scores, dim=-1)

    # Weighted sum of values
    output = torch.matmul(attention_weights, V)
    return output, attention_weights

class CrossAttention(nn.Module):
    def __init__(self, embed_size, text_embed_size):
        super(CrossAttention, self).__init__()
        self.embed_size = embed_size
        self.text_embed_size = text_embed_size

        # Linear layers for Q, K, V
        self.query = nn.Linear(embed_size, embed_size)
        self.key = nn.Linear(text_embed_size, embed_size)
        self.value = nn.Linear(text_embed_size, embed_size)

        # Final linear layer after concatenating heads
        self.fc_out = nn.Linear(embed_size, embed_size)

    def forward(self, target, source):

        Q = self.query(target)
        K = self.key(source)
        V = self.value(source)

        # Perform attention calculation (self or cross)
        out, _ = cross_attention(Q, K, V)
        return self.fc_out(out)

class CrossAttentionWithResidual(nn.Module):
    def __init__(self, embed_size, text_embed_size):
        super(CrossAttentionWithResidual, self).__init__()
        self.embed_dim = embed_size
        self.attention = CrossAttention(embed_size, text_embed_size)
        self.norm = nn.LayerNorm(embed_size)
        self.dropout = nn.Dropout(0.1)

    def forward(self, target, source):
        batch_size, channels, height, width = target.shape

        seq_len = height * width  # 4 * 4 = 16
        target = target.permute(0, 2, 3, 1).reshape(batch_size, seq_len, self.embed_dim)

        attention_out = self.attention(target, source)
        # Add residual connection and layer normalization
        out = self.norm(target + self.dropout(attention_out))

        attn_output = out.view(batch_size, height, width, self.embed_dim).permute(0, 3, 1, 2)
        return attn_output

In [18]:
class ELUnet(nn.Module):
    def __init__(self,in_channels,out_channels,text_embed_size,n:int = 8) -> None:
        """
        Construct the Elu-net model.
        Args:
            in_channels: The number of color channels of the input image. 0:for binary 3: for RGB
            out_channels: The number of color channels of the input mask, corresponds to the number
                            of classes.Includes the background
            n: Channels size of the first CNN in the encoder layer. The bigger this value the bigger
                the number of parameters of the model. Defaults to n = 8, which is recommended by the
                authors of the paper.
        """
        super().__init__()

        self.const = n

        # ------ Input convolution --------------
        self.in_conv = DoubleConv(in_channels,n)
        # -------- Encoder ----------------------
        self.down_1 = DownSample(n, 2*n)
        self.down_2 = DownSample(2*n, 4*n)
        self.down_3 = DownSample(4*n, 8*n)
        self.down_4 = DownSample(8*n, 16*n)

        # ------ Cross Attention --------------
        self.model_attention = CrossAttentionWithResidual(16*n, text_embed_size)

        # -------- Upsampling ------------------
        self.up_1024_512 = UpSample(16*n, 8*n, 2)

        self.up_512_64 = UpSample(8*n, n, 8)
        self.up_512_128 = UpSample(8*n, 2*n, 4)
        self.up_512_256 = UpSample(8*n, 4*n, 2)
        self.up_512_512 = UpSample(8*n, 8*n, 0)

        self.up_256_64 = UpSample(4*n, n, 4)
        self.up_256_128 = UpSample(4*n, 2*n, 2)
        self.up_256_256 = UpSample(4*n, 4*n, 0)

        self.up_128_64 = UpSample(2*n, n, 2)
        self.up_128_128 = UpSample(2*n, 2*n, 0)

        self.up_64_64 = UpSample(n, n, 0)

        # ------ Decoder block ---------------
        self.dec_4 = DoubleConv(2*8*n,8*n)
        self.dec_3 = DoubleConv(3*4*n,4*n)
        self.dec_2 = DoubleConv(4*2*n,2*n)
        self.dec_1 = DoubleConv(5*n,n)
        # ------ Output convolution

        self.out_conv = OutConv(n,out_channels)


    def forward(self, x, text_embeddings):

        x = self.in_conv(x) # 64

        # ---- Encoder outputs
        x_enc_1 = self.down_1(x) # 128
        x_enc_2 = self.down_2(x_enc_1) # 256
        x_enc_3 = self.down_3(x_enc_2) # 512
        x_enc_4 = self.down_4(x_enc_3) # 1024

        # ------ Cross Attention layer
        attention_result = self.model_attention(x_enc_4, text_embeddings)

        # ------ decoder outputs
        x_up_1 = self.up_1024_512(attention_result)
        x_dec_4 = self.dec_4(torch.cat([x_up_1,self.up_512_512(x_enc_3)],dim=1))

        x_up_2 = self.up_512_256(x_dec_4)
        x_dec_3 = self.dec_3(torch.cat([x_up_2,
            self.up_512_256(x_enc_3),
            self.up_256_256(x_enc_2)
            ],
        dim=1))

        x_up_3 = self.up_256_128(x_dec_3)
        x_dec_2 = self.dec_2(torch.cat([
            x_up_3,
            self.up_512_128(x_enc_3),
            self.up_256_128(x_enc_2),
            self.up_128_128(x_enc_1)
        ],dim=1))

        x_up_4 = self.up_128_64(x_dec_2)
        x_dec_1 = self.dec_1(torch.cat([
            x_up_4,
            self.up_512_64(x_enc_3),
            self.up_256_64(x_enc_2),
            self.up_128_64(x_enc_1),
            self.up_64_64(x)
        ],dim=1))

        return self.out_conv(x_dec_1)

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
elunet = ELUnet(in_channels=3, out_channels=1, text_embed_size=256)
elunet = elunet.to(device)

In [20]:
temp1 = torch.rand(1, 3, 256, 256).to(device) # Add a batch dimension
temp2 = torch.rand(1, 256).to(device) # Add a batch dimension

In [21]:
result = elunet(temp1, temp2)
result.shape

torch.Size([1, 1])

# **Distil Bert Tokenizer**

In [22]:
from sklearn.decomposition import PCA

# model_name = 'aubmindlab/bert-base-arabertv2'

model_name = 'distilbert-base-multilingual-cased'
fixed_length = 256

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    cache_dir=None,
    use_fast=True,
    revision="main",
    use_auth_token=None,
)

# text_model = AutoModel.from_pretrained(
#     model_name,
#     cache_dir=None,
#     revision="main",
#     use_auth_token=None,
# ).eval()

def tokenize_fixed_length(text):
    """
    Tokenize Arabic text to a fixed length with non-zero input_ids.
    Repeats non-special tokens if the text is too short.
    """
    encoding = tokenizer(
        text,
        padding='max_length',  # No padding initially
        truncation=True,  # Truncate if too long
        max_length=fixed_length,  # Ensure we don't exceed fixed_length
        return_tensors="pt"
    )

    # Extract input_ids and attention_mask
    input_ids = encoding['input_ids'][0]  # Shape: (seq_len,)
    attention_mask = encoding['attention_mask'][0]

    # Get non-special tokens (exclude [CLS] and [SEP])
    non_special_ids = input_ids[attention_mask == 1][1:-1]  # Exclude [CLS] (101) and [SEP] (102)

    if len(non_special_ids) == 0:
        new_input_ids = [101] + [100] * (fixed_length - 2) + [102]
    else:
        repeat_count = (fixed_length - 2) // len(non_special_ids) + 1
        repeated_ids = (non_special_ids.repeat(repeat_count)[:fixed_length - 2]).tolist()
        new_input_ids = [101] + repeated_ids + [102]
        new_input_ids = new_input_ids + [new_input_ids[-2]] * (fixed_length - len(new_input_ids))

    new_input_ids = new_input_ids[:fixed_length]
    new_attention_mask = [1] * fixed_length

    return {
        'input_ids': torch.tensor([new_input_ids]),
        'attention_mask': torch.tensor([new_attention_mask])
    }

def tokenize_fixed_length_2(text):
    encoding = tokenizer(
        text,
        padding='max_length',  #No padding initially
        truncation=True,  # Truncate if too long
        max_length=fixed_length,  # Ensure we don't exceed fixed_length
        return_tensors="pt"
    )

    # Extract input_ids and attention_mask
    input_ids = encoding['input_ids'][0]  # Shape: (seq_len,)
    attention_mask = encoding['attention_mask'][0]

    # Get non-special tokens (exclude [CLS] and [SEP])
    non_special_ids = input_ids[attention_mask == 1][1:-1]  # Exclude [CLS] (101) and [SEP] (102)

    if len(non_special_ids) == 0:
        new_input_ids = [101] + [100] * (fixed_length - 2) + [102]
    elif len(non_special_ids) > fixed_length:
        pca = PCA(n_components=fixed_length-2)
        result = pca.fit_transform(non_special_ids)
        new_input_ids = [101] + result + [102]
    else:
        repeat_count = (fixed_length - 2) // len(non_special_ids) + 1
        repeated_ids = (non_special_ids.repeat(repeat_count)[:fixed_length - 2]).tolist()
        new_input_ids = [101] + repeated_ids + [102]
        new_input_ids = new_input_ids + [new_input_ids[-2]] * (fixed_length - len(new_input_ids))

    new_input_ids = new_input_ids[:fixed_length]
    new_attention_mask = [1] * fixed_length

    return {
        'input_ids': torch.tensor([new_input_ids]),
        'attention_mask': torch.tensor([new_attention_mask])
    }

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [23]:
short_text = "مرحبا"
vector1 = tokenize_fixed_length_2(short_text)
print(vector1['input_ids'][0])

tensor([ 101, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,
         195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,
         195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,
         195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,
         195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,
         195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,
         195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,
         195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,
         195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,
         195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,
         195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,
         195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,  195, 9950,
         195, 9950,  195, 9950,  195, 99

# **Custom Dataset**

In [24]:
class CustomImageDataset(Dataset):
    def __init__(self, df, image_dir, transform=None):
        self.df = df  # Use provided train_df DataFrame
        self.image_dir = image_dir
        self.transform = transform
        self.skipped_samples = []  # Initialize skipped_samples list

        #print(f"Labels in DataFrame: {self.df['hate_label'].unique()}")
        print(f"Loaded {len(self.df)} rows")

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        id = row['id']
        image_name = row['img_path']
        image_path = os.path.join(self.image_dir, image_name.lstrip('./'))
        label = torch.tensor([row['hate_label']])
        text = row.get('processed_text', '')
        embedding = tokenize_fixed_length_2(text)

        try:
            image = Image.open(image_path).convert('RGB')
            if self.transform:
                image = self.transform(image)
        except Exception as e:
            print(f"Error loading image {row['id']} from {image_path}: {e}")
            self.skipped_samples.append({
                'index': row['id'],
                'img_path': image_path,
                'reason': f'Error loading image: {e}'
            })
            image = torch.zeros(3, 256, 256)  # Placeholder for failed images
        # return {
        #     'id': id,
        #     'image': image,
        #     'embedding': embedding
        # }

        return image, embedding, label  # Return only image and label

def _convert_image_to_rgb(image):
    return image.convert("RGB")

In [25]:
size = 256
transform = torch_transforms.Compose([
        torch_transforms.Resize(size, interpolation=InterpolationMode.BICUBIC),
        torch_transforms.CenterCrop(size),
        _convert_image_to_rgb,
        torch_transforms.ToTensor(),
        torch_transforms.Normalize([0.5], [0.5])
    ])

image_dir = "/content/drive/MyDrive/Prop2Hate-Meme/"  # Update to your image folder

target_dataset = CustomImageDataset(train_df, image_dir, transform=transform)

print(f"Loaded {len(target_dataset)} images")

Loaded 2143 rows
Loaded 2143 images


In [26]:
target_dataset[1][0].shape

torch.Size([3, 256, 256])

In [27]:
from torch.utils.data import DataLoader

In [28]:
def compute_class_weights(dataset):
    """
    Compute pos_weight for BCEWithLogitsLoss based on class distribution.

    Args:
        dataset: Training dataset with labels
    Returns:
        pos_weight: Scalar for BCEWithLogitsLoss
    """
    labels = [dataset[i][2].item() for i in range(len(dataset))]  #  label is at index 2
    num_positive = sum(labels)
    num_negative = len(labels) - num_positive
    pos_weight = num_negative / max(num_positive, 1)  # Avoid division by zero
    return torch.tensor([pos_weight], dtype=torch.float)

In [29]:
def get_weighted_sampler(dataset):
    """
    Create a WeightedRandomSampler for balanced sampling.

    Args:
        dataset: Training dataset with labels
    Returns:
        sampler: WeightedRandomSampler for DataLoader
    """
    labels = [dataset[i][2].item() for i in range(len(dataset))]
    class_counts = Counter(labels)
    weights = [1.0 / class_counts[label] for label in labels]
    sampler = WeightedRandomSampler(weights, len(weights), replacement=True)
    return sampler

# **Training**

In [30]:
# Set random seeds for reproducibility
torch.manual_seed(42)
random.seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
# frames = [train_df, dev_df]
# df = pd.concat(frames, ignore_index=True)
train_dataset = CustomImageDataset(train_df, image_dir, transform=transform)
dev_dataset = CustomImageDataset(dev_df, image_dir, transform=transform)
test_dataset = CustomImageDataset(test_df, image_dir, transform=transform)

Loaded 2143 rows
Loaded 312 rows
Loaded 606 rows


In [31]:
def train_model(model,
                train_dataset,
                val_dataset,
                criterion,
                optimizer,
                num_epochs,
                device,
                patience=3,
                batch_size=16,
                min_delta=0.001,
                use_weighted_sampler=False):

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Create DataLoader with optional weighted sampling
    if use_weighted_sampler:
        sampler = get_weighted_sampler(train_dataset)
        train_loader = DataLoader(train_dataset, batch_size=32, sampler=sampler)
    else:
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    # DataLoader for batching
    #train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=True)
    # criterion = nn.MSELoss()
    # optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate, weight_decay=1e-5)

    model.to(device)
    best_val_loss = float('inf')
    patience_counter = 0
    best_model_path = os.path.join(output_dir, 'elunet_best_model_distilBert.pth')

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_total = 0

        for batch in tqdm.tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
          images, texts_dict, labels = batch # Changed variable name to reflect that it's a dictionary
          images = images.to(device)
          texts = texts_dict['input_ids'].to(device).float() # Extract 'input_ids' and move to device, convert to float
          labels = labels.to(device).float()  # Assuming binary labels (0 or 1)

          optimizer.zero_grad()
          outputs = model(images, texts)  # Removed .squeeze()
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          train_loss += loss.item() * images.size(0)
          train_total += images.size(0)
        avg_train_loss = train_loss / train_total
        print(f"Epoch {epoch+1}/{num_epochs} - Training Loss: {avg_train_loss:.4f}")

        # Validation phase
        model.eval()
        val_loss = 0.0
        val_total = 0
        with torch.no_grad():
          for batch in tqdm.tqdm(val_loader, desc="Validation"):
            images, texts_dict, labels = batch # Changed variable name to reflect that it's a dictionary
            images = images.to(device)
            texts = texts_dict['input_ids'].to(device).float() # Extract 'input_ids' and move to device, convert to float
            labels = labels.to(device).float()
            outputs = model(images, texts) # Removed .squeeze()
            loss = criterion(outputs, labels)
            val_loss += loss.item() * images.size(0)
            val_total += images.size(0)

        avg_val_loss = val_loss / val_total
        print(f"Epoch {epoch+1}/{num_epochs} - Validation Loss: {avg_val_loss:.4f}")

        # Early stopping
        if avg_val_loss < best_val_loss - min_delta:
          best_val_loss = avg_val_loss
          patience_counter = 0
          # Save model weights
          torch.save(model.state_dict(), best_model_path)
          print(f"Saved best model weights to: {best_model_path}")
        else:
          patience_counter += 1
          print(f"No improvement in validation loss. Patience counter: {patience_counter}/{patience}")
          if patience_counter >= patience:
            print(f"Early stopping triggered after epoch {epoch+1}")
            break

    # Load best model weights before returning
    model.load_state_dict(torch.load(best_model_path))
    print(f"Loaded best model weights from: {best_model_path}")
    return model

    # for epoch in range(num_epochs):
    #     total_loss = 0
    #     total = 0
    #     correct = 0
    #     for batch in train_loader:
    #       #print('1')
    #       images, text_embedding, target = batch

    #       images = images.to(device).float()
    #       text_embedding = text_embedding['input_ids'][0].to(device).float()
    #       target = target.to(device).float()

    #       # Forward pass
    #       outputs = model(images, text_embedding)
    #       loss = criterion(outputs, target)

    #       # Backward pass and optimization
    #       optimizer.zero_grad()
    #       loss.backward()
    #       optimizer.step()

    #       predicted = (torch.sigmoid(outputs) >= 0.5).float()
    #       correct += (predicted == target.view(-1, 1)).sum().item()
    #       total += target.size(0)
    #       total_loss += loss.item()

    #     # Print epoch loss
    #     avg_loss = total_loss / len(train_loader)
    #     accuracy = 100 * correct / total
    #     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%')

# Train the model
#train_model(model=elunet, train_dataset=target_dataset, batch_size=16, num_epochs=15, learning_rate=0.001)

In [32]:
def evaluate_metrics(model, test_dataset, device):
    """
    Evaluate the model on the test set and save classification report to JSON.

    Args:
        model: Trained ELUnet model
        test_loader: DataLoader for test data
        device: Device to run evaluation on ('cuda' or 'cpu')
    """
    model.eval()
    all_preds = []
    all_labels = []
    all_data = []
    test_loader = DataLoader(dataset=test_dataset, batch_size=4, shuffle=False) # Set shuffle to False
    with torch.no_grad():
        for batch_idx, batch in enumerate(tqdm.tqdm(test_loader, desc="Testing")): # Enumerate to get batch index
            images, texts_dict, labels = batch
            images = images.to(device)
            texts = texts_dict['input_ids'].to(device).float() # Extract 'input_ids' and move to device, convert to float
            labels = labels.to(device).float()

            outputs = model(images, texts).squeeze()
            preds = (outputs > 0.5).float()  # Threshold for binary classification
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            # Collect dataset-specific fields (id, text, img_path, hate_label) and correctness
            for i in range(len(labels)):
                # Calculate the original index in the dataset
                original_idx = batch_idx * test_loader.batch_size + i
                if original_idx < len(test_dataset): # Ensure index is within dataset bounds
                    sample = test_dataset[original_idx]
                    # Check if sample is not None (in case image loading failed)
                    if sample is not None:
                        true_label = int(labels[i].cpu().numpy())
                        pred_label = int(preds[i].cpu().numpy())
                        all_data.append({
                            'id': test_dataset.df.iloc[original_idx]['id'], # Access id from original dataframe
                            'text': test_dataset.df.iloc[original_idx]['text'], # Access text from original dataframe
                            'img_path': test_dataset.df.iloc[original_idx]['img_path'], # Access img_path from original dataframe
                            'hate_label': true_label,  # True label
                            'predicted_label': pred_label,  # Predicted label
                            'correct': true_label == pred_label  # True if prediction matches true label, False otherwise
                        })

    # Calculate metrics
    accuracy = accuracy_score(all_labels, all_preds)
    class_report = classification_report(all_labels, all_preds, output_dict=True)
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    precision, recall, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')

    # Save classification report to JSON
    report_path = os.path.join(output_dir, 'classification_report_with_weight_distilBert.json')
    with open(report_path, 'w') as f:
        json.dump(class_report, f, indent=4)

    # Save predictions, dataset columns, and correctness to CSV
    predictions_path = os.path.join(output_dir, 'predictions_with_labels_with_weight_distilBert.csv')
    df = pd.DataFrame(all_data)
    df.to_csv(predictions_path, index=False, encoding='utf-8-sig')

    # Print metrics
    print(f"Test Accuracy: {accuracy:.4f}")
    print(f"Macro F1 Score: {macro_f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")
    print(f"Classification Report saved to: {report_path},{predictions_path}")

    return accuracy, macro_f1, precision, recall, f1

In [33]:
train_dataset[1][2]

tensor([0])

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = elunet.to(device)
# pos_weight = compute_class_weights(train_dataset)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)  # Adjust learning rate as needed
num_epochs = 10
patience = 3  # Number of epochs to wait for improvement
min_delta = 0.001  # Minimum improvement in validation loss

In [35]:
num_epochs = 20
model = elunet.to(device)

In [36]:
# Train the model
model = train_model(model, train_dataset, dev_dataset, criterion, optimizer, num_epochs, device, patience, batch_size=16, min_delta=min_delta, use_weighted_sampler=False)

# Evaluate the model
#test_loader = DataLoader(dataset=test_dataset, batch_size=16, shuffle=True)


Epoch 1/20: 100%|██████████| 67/67 [01:45<00:00,  1.57s/it]


Epoch 1/20 - Training Loss: 0.8662


Validation: 100%|██████████| 20/20 [00:07<00:00,  2.59it/s]


Epoch 1/20 - Validation Loss: 0.8435
Saved best model weights to: /content/drive/MyDrive/Prop2Hate-Meme/Result/Elunet/WithOut_Diff/elunet_best_model_distilBert.pth


Epoch 2/20: 100%|██████████| 67/67 [01:33<00:00,  1.40s/it]


...

Epoch 19/20 - Validation Loss: 0.6953
No improvement in validation loss. Patience counter: 1/3


Epoch 20/20: 100%|██████████| 67/67 [01:21<00:00,  1.22s/it]


Epoch 20/20 - Training Loss: 0.6949


Validation: 100%|██████████| 20/20 [00:08<00:00,  2.38it/s]

Epoch 20/20 - Validation Loss: 0.6949
No improvement in validation loss. Patience counter: 2/3
Loaded best model weights from: /content/drive/MyDrive/Prop2Hate-Meme/Result/Elunet/WithOut_Diff/elunet_best_model_distilBert.pth


In [37]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
elunet = ELUnet(in_channels=3, out_channels=1, text_embed_size=256)
model = elunet
model.load_state_dict(torch.load('/content/drive/MyDrive/Prop2Hate-Meme/Result/Elunet/WithOut_Diff/elunet_best_model_distilBert.pth', weights_only=True))
model.to(device)

ELUnet(
  (in_conv): DoubleConv(
    (double_conv): Sequential(
      (0): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (4): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
    )
  )
  (down_1): DownSample(
    (down_sample): Sequential(
      (0): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (1): DoubleConv(
        (double_conv): Sequential(
          (0): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
          (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU()
          (3): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=same)
          (4): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=Tr

# **Evaluation**

In [38]:

test_loader = DataLoader(dataset=test_dataset, batch_size=4, shuffle=True)

In [39]:
evaluate_metrics(model, test_dataset, device)

Testing:   0%|          | 0/152 [00:00<?, ?it/s]/tmp/ipython-input-2603053714.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  true_label = int(labels[i].cpu().numpy())
Testing: 100%|██████████| 152/152 [00:27<00:00,  5.44it/s]

Test Accuracy: 0.7459
Macro F1 Score: 0.4272
Precision: 0.3729
Recall: 0.5000
F1 Score: 0.4272
Classification Report saved to: /content/drive/MyDrive/Prop2Hate-Meme/Result/Elunet/WithOut_Diff/classification_report_without_weight_araBert.json,/content/drive/MyDrive/Prop2Hate-Meme/Result/Elunet/WithOut_Diff/predictions_with_labels_without_weight_distilBert.csv



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/

(0.7458745874587459,
 0.42722117202268434,
 0.37293729372937295,
 0.5,
 0.42722117202268434)

In [40]:
import pandas as pd

# Load the predictions from the CSV file
predictions_df = pd.read_csv('/content/drive/MyDrive/Prop2Hate-Meme/Result/Elunet/WithOut_Diff/predictions.csv')

# Check if the 'id' columns are the same
if predictions_df['id'].equals(test_df['id']):
    print("The 'id' columns are in the same order.")
else:
    print("The 'id' columns are not in the same order.")
    # Find and print the differences
    for i, (pred_id, test_id) in enumerate(zip(predictions_df['id'], test_df['id'])):
        if pred_id != test_id:
            print(f"Mismatch at index {i}: prediction_id='{pred_id}', test_id='{test_id}'")

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Prop2Hate-Meme/Result/Elunet/WithOut_Diff/predictions.csv'